# Zero To Hero Big Data Prepration
Taking Advantage of Cloud Technologies to Create Big Data Solutions
    
    AUTHOR: Dr. Roy Jafari 

# Chapter 3: Using the right data strucutres 

## Challenge 4 - tables, matrices or graphs?

In this challenge, we will be using the shortest path problem and Dijkstra’s algorithm that can find the optimal solution to the problem in O(m) complexity, m being the number of edges of the graph in the shortest path problem. What is the graph of the shortest path problem or its number of edges that’s a great question? We will understand those soon. Answer the questions and perform the tasks given in the following.

1.	Consider the weighted directional graph in *Figure 8.5*. Assume we are standing in the node ③ and we wish to go to node ④. The number on the edges is the cost we have to pay to use the edge to travel to the other node. What is the minimum cost we can pay to go from node ③ to node ④? What is the path that takes us from node ③ to node ④ with the minimum cost? I have put the graph on a single page that you can download that print out for your convenience. You may download the file using this link: https://packt-data-prep-workshop.s3.eu-west-1.amazonaws.com/Figure8_5_d.pdf.


**Answer**: The point here is not the correct final answer but having tried to solve the problem by yourself once. 

2.	You just solved the shortest path problem. What is the number of edges on the graph you just solved? What is the number of nodes? 

**Answer**: The number of edges are 11, and the number of nodes are 7.

3.	In this step, you will watch a YouTube video. The video is a visual introduction to Dijkstra’s algorithm. The link to the YouTube video is https://youtu.be/iK5fNRWxucA. After you watched the video, in words, explain how Dijkstra’s algorithm work. Specifically, mention what is the task that the algorithm iteratively repeats. What is the maximum number of times that the iteration has to happen? What is the computational complexity of Dijkstra’s Algorithm?

**Answer**: The algorithm starts from the source and once travels to each node from the source and updates the distance it takes to get to the note if it is smaller that the ones we had seen before. What happens iteratively is considering the possible move from the current node to one of its adjacent nodes to see if that travel should be part of the shortest path. The computational complexity of the algorithm will be *O(m)*, *m* being the number of edges.

4.	Now, we will implement *Dijkstra’s algorithm* in python and do some experimentation with it. First, we will have to define a graph class so we can introduce the graph to our computer. Earlier in the chapter, we discussed three different data structures we could encode graphs with: a table, a matrix, and a list. We implemented those under the names `GraphLE`, `GraphAM`, and `GraphAL`. We will experiment with all of those in this challenge. So please go ahead, and redefine those classes, we will be using all of them in this challenge.

In [4]:
import numpy as np
import pandas as pd

class GraphLE(object):
    def __init__(self,n_nodes=None):
        self.edge_df = pd.DataFrame(
            columns = ['n1','n2','w']
        )
        self.n_nodes = n_nodes
    def add_edge(self,n1,n2,w=1):
        add_df = pd.DataFrame(
            {'n1': n1, 'n2': n2, 'w': w},
            index=[len(self.edge_df)]
        )
        self.edge_df = pd.concat([self.edge_df,add_df])
    def print_all_adj_list(self):
        print(self.edge_df)
    def get_edge_from(self,node):
        BM = self.edge_df.n1 == node
        return [(r.n1,r.n2,r.w) for i,r in 
               self.edge_df[BM].iterrows()]

In [5]:
class GraphAM(object):
    def __init__(self,n_nodes):
        self.n_nodes = n_nodes
        self.adj_matrix = (
            np.zeros([n_nodes,n_nodes])
            .astype(int)
        )
    def add_edge(self,n1,n2,w=1):
        self.adj_matrix[n1,n2] = w
    def print_adj_matrix(self):
        print(self.adj_matrix)
    def get_edge_from(self,n1):
        return [(n1,n2,w) for n2,w in
                enumerate(self.adj_matrix[n1,:]) if w>0]

In [6]:
class GraphAL(object):
    def __init__(self,n_nodes):
        self.n_nodes = n_nodes
        self.adj_list = {
            node: list() for node in range(self.n_nodes)
        }
    def add_edge(self,n1,n2,w=1):
        (self
         .adj_list[n1]
         .append((n2,w))
        )
    def print_all_adj_list(self):
        for key in self.adj_list.keys():
            print(key,self.adj_list[key])
    def get_edge_from(self,n1):
        return [(n1,n2,w) for n2,w in self.adj_list[n1]]

5.	The following code uses `GraphLE` to hold the weighted directional graph in *Figure 8.5*. Run the code so the graph is encoded on your computer. We will use the encoded graph to test our implementation of *Dijkstra’s algorithm*. After you’ve run the following code, run `le_graph.print_all_adj_list()` to see how GraphLe has encoded the graph. 

In [7]:
edge_list = [(3,0,5),(3,1,2),(1,0,7),(0,2,1),(2,5,9),(6,2,3),(5,6,3),(5,1,2),(5,4,9),(4,1,3),(4,5,8)]
n_nodes = 7
le_graph = GraphLE(n_nodes)
for n1,n2,w in edge_list:
    le_graph.add_edge(n1, n2, w)

In [8]:
le_graph.print_all_adj_list()

   n1 n2  w
0   3  0  5
1   3  1  2
2   1  0  7
3   0  2  1
4   2  5  9
5   6  2  3
6   5  6  3
7   5  1  2
8   5  4  9
9   4  1  3
10  4  5  8


6.	From the file *Chapter8.ipynb* under folder ch8 in the GitHub Repository of the book find the chunk of code that defines the function `dijkstra()`, and run the code to define the function. The reason that the code is not provided here is that it is too long. Before moving on to the next step, I highly recommend that you also study the code to figure out how the code has implemented the algorithm. Please pay attention that the internal variables used in the code, naming `min_dist`, `finalized`, and `final_path`, use the same wordings as the ones that you saw in the YouTube video of step 2. This is intentional to help you understand the implementation easier.

In [9]:
import pandas as pd
import numpy as np

def dijkstra(graph,source):
    n_nodes = graph.n_nodes
    
    min_dist = pd.Series(float('inf'),index=range(n_nodes))
    min_dist[source] = 0
    
    finalized = pd.Series(False,index=range(n_nodes))
    final_path = {node:None for node in range(n_nodes)}
    
    while sum(finalized) <n_nodes:
        node = min_dist[finalized==False].idxmin()
        for n1,n2,w in graph.get_edge_from(node):
            
            new_w = min_dist.iloc[n1] + w 
            
            if(new_w< min_dist.iloc[n2]):
                min_dist.iloc[n2] = new_w
                final_path[n2] = n1
        
        finalized[node] = True
        
    shortest_paths = {}
    
    for i in range(n_nodes):
        path = []
        dest = i

        path.append(dest)

        while final_path[dest] != None:
            path.append(final_path[dest])
            dest = final_path[dest]
        
        shortest_paths[i] = path[::-1]

    return min_dist, shortest_paths

7.	The following code uses the function `dijkstra()` to solve the shortest path problem given the graph `le_graph` and from the source node ③. Run the code, and study its printouts. Are the printouts consistent with the solutions that were found for the same graph in the YouTube video in step 2?

In [10]:
source=3
min_path, shortest_path =  dijkstra(le_graph,source)
print(min_path)
print(shortest_path)

0     5.0
1     2.0
2     6.0
3     0.0
4    24.0
5    15.0
6    18.0
dtype: float64
{0: [3, 0], 1: [3, 1], 2: [3, 0, 2], 3: [3], 4: [3, 0, 2, 5, 4], 5: [3, 0, 2, 5], 6: [3, 0, 2, 5, 6]}


**Answer**: The answers are exactly the same.

8.	So far in this challenge, we only developed what we need to perform experiments that could tell us which data structure is superior to encode graphs.  The following code will randomly generate a big graph. Run the following code to create the big graph.

In [14]:
import numpy as np
n_nodes = 10000
n_edges = 50000
edges = np.random.randint(0,n_nodes,[n_edges,2])
weights = np.random.randint(1,25,n_edges)
edge_list = [(edges[i,0],edges[i,1],weights[i]) for i in range(n_edges)]


9.	 We will be experimenting with the three types of data structures that we can use to hold a graph in a computer’s RAM and compare them based on two criteria 1) `FillingTime`: how long it takes for us to fill up the data structure, 2) `RunTime`: how long it takes for us to solve the shortest path problem using Dijkstra’s algorithm. The following code creates a DataFrame that we will use to record these metrics. Run the following code to create the DataFrame and then study it.

In [15]:
datastrcutre_options = {'table': GraphLE,
                       'matrix': GraphAM,
                       'graph': GraphAL}

result_df = pd.DataFrame(
    index = datastrcutre_options,
    columns = ['FillingTime','RunTime']
)
print(result_df)

       FillingTime RunTime
table          NaN     NaN
matrix         NaN     NaN
graph          NaN     NaN


10.	 The following code creates three different graphs using the three classes `GraphLE`, `GraphAM`, and `GraphAL`. For each encoded graph, first, it records the time it takes for your computer’s CPU to fill up the big graph that was created in step 9, after that, it records the time your CPU takes to solve the shortest path problem from node ④. You may ask, why node ④? It’s an arbitrary choice, you may choose any other node of the randomly generated graph. Running the following code may take a few minutes to complete, once it is done study its recorded times on `result_df`. 

In [16]:
import time
for ds_name, ds_class in datastrcutre_options.items():
    t0 = time.time()
    graph = ds_class(n_nodes)
    for s,d,w in edge_list:
        graph.add_edge(s, d, w)
    
    result_df.at[ds_name,'FillingTime'] = time.time() - t0
    
    t0 = time.time()
    min_dist, shortest_path =  dijkstra(graph,4)
    result_df.at[ds_name,'RunTime'] = time.time() - t0
print(result_df)

       FillingTime     RunTime
table    47.109389  101.564448
matrix    0.315183   24.215499
graph     0.104828    8.725793


11. Looking at the result of the experimentations in `result_df`, which data structure led to the fastest filling up of the graph? Which one led to the fastest solution performance of *Dijkstra’s algorithm*? Looking at the code to create classes of `GraphLE`, `GraphAM`, and `GraphAL` explains why each data structure leads to faster or slower performance.

**Answer**: For both `FillingTime` and `RunTime`, *graph* is the fastest and *table* is the slowest.

**Why table is slow for `FillingTime`?** The reason is that every time a new edge is introduced to the graph (`.add_edge()`), the graph must update the table(dataframe)'s structure and that is expensive for the CPU.

**Why graph is fast for `FillingTime`?** The reason is that the `.add_edge()` function in GraphAL will find the list of adjacent nodes from the `adj_list` dictionary and append the new adjacent nodes to the list. Because the dictionary is indexed efficiently and appending is a cheap CPU action, the `FillingTime` for the graph is fasters.

**Why matrix is not as fast as the graph in `FillingTime`?**: The matrix implementation is slower than the graph because every `.add_edge()` run for GraphAM CPU must find the correct cell to update from 100,000,000 cells and that's expensive for the CPU.

**Why table is slow for `RunTime`?** *Dijkstra’s algorithm* only uses the function `.get_edge_from()` and that function in `GraphLE` involves using a Boolean mask to find the edges that start with a node. For a DataFrame of 50000 rows, this is not CPU cheap.

**Why graph is fast for `RunTime`?** *Dijkstra’s algorithm* only uses the function `.get_edge_from()` and that function in `GraphAL` only needs to find the list of adjacent nodes from the `adj_list` dictionary, and that's very easy for CPU. 

**Why matrix is not as fast as the graph in `RunTime`?**: Dijkstra’s algorithm* only uses the function `.get_edge_from()` and that function in `GraphAM` needs to search 100,000,000 cells to find a handful of hits, and that's not cheap for the CPU.


12.	 All in all, which data structure is superior when we have to work with graphs?

**Answer**: 
When we work with graph data, the information that is used in runtime is to know the adjacent list to a node, and `GraphAL` provides that access to the information much easier as we experienced in this challenge. Therefore, the absolute superior not just in terms of `RunTime` but the time it took to introduce the data RAM (`FillingTime`) is `GraphAL`.


13.	Come up with two real-life cases in which we can use GraphAL to keep the case’s information more efficiently. 

**Answer**:

The real-life cases are many. Here are two:

- **Social Media**: The nodes are people, and the edges are the relationships between the people. The edges can be directional to model "following" and could be non-directional to model "connection" or being "friends". The edges could be weighted on not based on the use case, for instance, the weight could be the number of messages people send to each other.

- **Policing Online Financial Transactions**: The nodes are bank accounts, the edges are transactions between the bank accounts, and the weights are the amount of money in the transaction. These graphs can be used to detect circles in financial transactions that are used for money laundering.